# Arm Assembly

Use CSYS to create a scene

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [2]:
import bumblebee as bb
from IPython.display import display

In [3]:
assm = bb.CSYS()
def home_view():
    assm.ax.set_xlim([-10, 10])
    assm.ax.set_ylim([-10, 10])
    assm.ax.set_zlim([-10,10])
home_view()
display(assm.plot())

## Arm

Add the arm to the scene

In [4]:
arm = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\Arm.stl", name="Arm", units='cm')
assm.add(arm)
arm.rotate_from_euler('z', -90, degrees=True)
arm.translate([-2, 0, 2])

Add the shoulder to the scene

In [5]:
shoulder = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\arm_shoulder.stl", name="ArmShoulder", units='cm')
assm.add(shoulder)
shoulder.rotate_from_euler('y', -90, degrees=True)
shoulder.rotate_from_euler('z', 180, degrees=True)
shoulder.translate([0,4,-4.564-1.75])

Add the elbow to the scene

In [6]:
def end_view():
    assm.ax.set_xlim([-10, 10])
    assm.ax.set_ylim([-150, -130])
    assm.ax.set_zlim([-10,10])
    
end_view()

In [7]:
elbow = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\arm_elbow.stl", name="ArmElbow", units='cm')
assm.add(elbow)
elbow.translate([0,-140.105 + 1.5,0])
elbow.rotate_from_euler('y', -90, degrees=True)
elbow.rotate_from_euler('z', 180, degrees=True)
elbow.translate([2,0,-2])

Check that everything looks right

In [8]:
def full_view():
    assm.ax.set_xlim([-75, 75])
    assm.ax.set_ylim([-150, 0])
    assm.ax.set_zlim([-75,75])
    
full_view()

Convert arm to an assembly

In [9]:
arm_assm = bb.RigidCollection(name='ArmAsm')

for body in [arm, elbow, shoulder]:
    arm_assm.add(body)
    assm.tree.remove_node(body)

arm_assm.bind(assm)
assm.tree.add_node(arm_assm)

Add frames representing the joint, and end of the link to be used in kinematic calcs

In [10]:
display(assm.plot())
home_view()

In [11]:
joint = bb.Frame(name='Joint')
arm_assm.add(joint)
joint.translate([0,1.25,-7.504-1.75])

In [12]:
end_view()

In [13]:
M = bb.Frame(name='M_pose')
arm_assm.add(M)
M.translate([0,-140.105-7.25,-2.375])

In [14]:
# due to the tangent point moving with angle this is only appx.
wire_pt = bb.Frame(name='WirePt')
arm_assm.add(wire_pt)
wire_pt.translate([0, -11.5+2.315, 0])

## Crown Assembly

In [15]:
home_view()

Add a frame to assist in locating the support joint

In [16]:
sup_joint = bb.Frame(name='SupportJoint')
assm.add(sup_joint)
sup_joint.translate([0, 2, 1.1+1.75])

Add the eccentric ring, this is needed to properly constrain the support beams.

In [17]:
eccentric = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\eccentric.stl", name="Eccentric", units='cm')
assm.add(eccentric)

In [18]:
eccentric.rotate_from_euler('y', -90, degrees=True)

In [19]:
eccentric.translate([0.5,2,1.1+1.75+0.5])

Add the veritical support beam

In [20]:
crown = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\crown.stl", name="Crown", units='cm')
assm.add(crown)

In [21]:
crown_assm = bb.RigidCollection(name='CrownAssm')
for body in [crown]:
    crown_assm.add(body)
    assm.tree.remove_node(body)

crown_assm.bind(assm)
assm.tree.add_node(crown_assm)

In [22]:
crown_joint = bb.Frame(name='CrownJoint')
crown_assm.add(crown_joint)
crown_joint.translate([3, 1.5, -0.75])

In [23]:
crown_M = bb.Frame(name='CrownM')
crown_assm.add(crown_M)
crown_M.translate([3, 40.697, 0])

In [24]:
crown_assm.tf = crown_joint.inv_tf @ sup_joint.tf
crown_assm.update()

Find the transform for the home position of the vertical support beam

In [25]:
import numpy as np
from scipy.spatial.transform import Rotation
cable1 = 157.875 #length of cable in tension
arm_len = np.sqrt(np.sum((M.tf[0:3,3]-sup_joint.tf[0:3,3])**2))
crown_len = np.sqrt(np.sum((crown_M.tf[0:3,3]-crown_joint.tf[0:3,3])**2))

a = cable1
b = arm_len
c = crown_len

alpha = np.arccos((b**2 + c**2 - a**2)/(2*b*c))

# get arm vector in yz plane
arm_vec = M.tf[1:3, 3] - sup_joint.tf[1:3,3]
arm_vec /= np.linalg.norm(arm_vec)
crown_vec_unit = (np.array([[np.cos(-alpha), -np.sin(-alpha)], [np.sin(-alpha), np.cos(-alpha)]]) @ arm_vec)
crown_vec = crown_vec_unit * crown_len

# plot the triangle
assm.ax.plot([0, 0, 0], [M.tf[1,3], crown_vec[0]+sup_joint.tf[1,3], sup_joint.tf[1,3]],[M.tf[2,3], crown_vec[1]+sup_joint.tf[2,3], sup_joint.tf[2,3]], 'k')

# get midpoint of rotated beam at joint
perp = np.array([[0,-1],[1,0]]) @ crown_vec_unit
mid = sup_joint.tf[1:3,3] + 0.75*perp

# get distances to midpoint from the rotated end point and joint pos
distM = np.sqrt(np.sum((crown_M.tf[1:3,3]-mid)**2))
distQ = np.sqrt(np.sum((sup_joint.tf[1:3,3]-mid)**2))

# get the angle between the midplane and the joint/endpoint line
gamma = np.arctan(distQ/distM)

# get the rotation angle relative to y axis

below_plane = np.arctan((M.tf[2,3]-sup_joint.tf[2,3])/(M.tf[1,3]-sup_joint.tf[1,3]))
crown_ang_from_y = np.pi - ((alpha - below_plane) + gamma)

R = Rotation.from_euler('x', crown_ang_from_y)

In [26]:
T=np.eye(4)
T[0:3,0:3] = R.as_matrix()
T[1:3,3] = crown_vec + sup_joint.tf[1:3,3]

rel_pose = crown_M.inv_tf @ crown_assm.tf

newT = T @ rel_pose
crown_assm.tf = newT.copy()
crown_assm.update()

## Wing Assembly

Create the wing in a new CSYS so that things don't get too cluttered. Will add back to full assembly later

In [27]:
assm2 = bb.CSYS()
def home_view():
    assm2.ax.set_xlim([-10, 10])
    assm2.ax.set_ylim([-10, 10])
    assm2.ax.set_zlim([-10,10])
home_view()
display(assm2.plot())

In [28]:
wing = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\wing.stl", name="wing", units='cm')
assm2.add(wing)

In [29]:
wing_shoulder = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\wing_shoulder.stl", name="WingShoulder", units='cm')
assm2.add(wing_shoulder)
wing_shoulder.rotate_from_euler('x', 90, degrees=True)
wing_shoulder.translate([0, 4, -4])

In [30]:
wing_cap = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\wing_cap.stl", name="WingCap", units='cm')
assm2.add(wing_cap)
wing_cap.rotate_from_euler('x', 90, degrees=True)
wing_cap.rotate_from_euler('z', 180, degrees=True)
wing_cap.translate([58.35, 0, -4])

In [31]:
wing_assm = bb.RigidCollection(name='WingAssm')
for body in [wing, wing_shoulder, wing_cap]:
    wing_assm.add(body)
    assm2.tree.remove_node(body)

wing_assm.bind(assm2)
assm2.tree.add_node(wing_assm)

In [32]:
wing_joint = bb.Frame(name='WingJoint')
wing_assm.add(wing_joint)
wing_joint.translate([-3.25, 2, -0.535])

In [33]:
wing_M = bb.Frame(name='WingM')
wing_assm.add(wing_M)
wing_M.translate([58.35+5.25, 2, -2])

Now add it to our original coordinate system

In [34]:
assm.add(wing_assm)
assm.plot()

In [35]:
wing_assm.rotate_from_euler('z', 90, degrees=True)

In [36]:
wing_M.rotate_from_euler('z', -90, degrees=True)

In [37]:
wing_vec = wing_M.tf[1:3,3] - wing_joint.tf[1:3,3]
wing_vec

array([66.85 , -1.465])

In [38]:
R = Rotation.from_euler('x', -95.86 + crown_ang_from_y*180/np.pi, degrees=True) # from the crown

In [39]:
loc = R.as_matrix() @ np.array([0, wing_vec[0], wing_vec[1]])

In [40]:
T=np.eye(4)
T[0:3,0:3] = R.as_matrix()
T[0:3,3] = loc + [0,2,1.1+1.75+0.5]

rel_pose = wing_M.inv_tf @ wing_assm.tf

newT = T @ rel_pose
wing_assm.tf = newT.copy()
wing_assm.update()

## Tail Assembly

In [41]:
assm3 = bb.CSYS()
def home_view():
    assm3.ax.set_xlim([-10, 10])
    assm3.ax.set_ylim([-10, 10])
    assm3.ax.set_zlim([-10,10])
home_view()
display(assm3.plot())

In [42]:
tail = bb.RigidBody(r"C:\Users\Elijah\Documents\PrinterAssembly\LowPoly\tail.stl", name="tail", units='cm')
assm3.add(tail)

In [43]:
tail.rotate_from_euler('xz', [90,90], degrees=True)

In [44]:
tail_assm = bb.RigidCollection(name='TailAssm')
for body in [tail]:
    tail_assm.add(body)
    assm3.tree.remove_node(body)

tail_assm.bind(assm3)
assm3.tree.add_node(tail_assm)

In [45]:
tail_joint = bb.Frame(name='TailJoint')
tail_assm.add(tail_joint)
tail_joint.translate([6.97/2, -1.933, 2.25])

In [46]:
tail_M = bb.Frame(name='TailM')
tail_assm.add(tail_M)
tail_M.translate([6.97/2, -1.933+83.461, 2.25])

In [47]:
assm.add(tail_assm)
assm.plot()

In [48]:
tail_vec = tail_M.tf[1:3,3] - tail_joint.tf[1:3,3]
R = Rotation.from_euler('x', -95.86 + crown_ang_from_y*180/np.pi -36.49, degrees=True) # from the crown
loc = R.as_matrix() @ np.array([0, tail_vec[0], tail_vec[1]])

In [49]:
T=np.eye(4)
T[0:3,0:3] = R.as_matrix()
T[0:3,3] = loc + sup_joint.tf[0:3, 3]

rel_pose = tail_M.inv_tf @ tail_assm.tf

newT = T @ rel_pose
tail_assm.tf = newT.copy()
tail_assm.update()

Save the assembly for later use

In [50]:
assm.ax.set_xlim([-100, 100])
assm.ax.set_ylim([-100, 100])
assm.ax.set_zlim([-100,100])
assm.plot()

In [51]:
assm.ax.plot(
    [M.tf[0,3], crown_M.tf[0,3], wing_M.tf[0,3], tail_M.tf[0,3]],
    [M.tf[1,3], crown_M.tf[1,3], wing_M.tf[1,3], tail_M.tf[1,3]],
    [M.tf[2,3], crown_M.tf[2,3], wing_M.tf[2,3], tail_M.tf[2,3]],
    'k'
)

In [52]:
full_assm = bb.RigidCollection(name='ArmAsm')
for body in [arm_assm, crown_assm, wing_assm, tail_assm]:
    full_assm.add(body)
full_assm.to_json('Arm')